In [1]:
# 加载向量数据库

import sys
sys.path.append("../")

In [2]:
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

/tmp/ipykernel_5009/576814584.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from zhipuai_embedding import ZhipuAIEmbeddings


In [3]:
import os

zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

In [13]:
# 定义Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../data_base/vector_db/chroma'

# 加载向量数据库
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)


In [14]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


In [15]:
# 测试一下加载的向量数据库，使用一个问题 query 进行向量检索
question = "什么是prompt engineering？"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


In [7]:
for i,doc in enumerate(docs):
    print(f"检索到的第{i}个内容：\n{doc.page_content}",end="\n-----------------------------------------------------\n")

检索到的第0个内容：
3
1.4.1
式(1.1) 和式(1.2) 的解释. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
4
第2 章模型评估与选择
5
2.1
经验误差与过拟合
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2
评估方法
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2.1
算法参数（超参数）与模型参数. . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.2.2
验证集. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.3
性能度量
-----------------------------------------------------
检索到的第1个内容：
39
4.5.1
图(4.10) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
4.5.2
图(4.11) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
第5 章神经网络
41
5.1
神经元模型. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2
感知机与多层网络
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2.1
式(5.1) 和式(5.2) 的推导. . . . . .

In [16]:
# 创建一个LLM
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0,openai_api_key=os.environ["OPENAI_API_KEY"],openai_api_base=os.environ["BASE_URL"])

llm.invoke("请你自我介绍一下自己！")

AIMessage(content='您好，我是一名智能助手，专门为您提供各种服务和信息。我具有丰富的知识和经验，可以帮助您解决问题和获取所需的信息。如果您有任何需求或疑问，都可以随时向我咨询，我会竭尽所能为您提供帮助。希望我们可以愉快地合作！如果有任何问题，请随时向我提问。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 20, 'total_tokens': 149, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bd2a0802-2272-49f7-b7e7-008c919268a5-0', usage_metadata={'input_tokens': 20, 'output_tokens': 129, 'total_tokens': 149, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [18]:
# 构建检索问答链
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)

In [19]:
# 创建一个基于模板的检索链
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt":QA_CHAIN_PROMPT}
)

In [20]:
# 检索问答链效果测试
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

In [21]:
# 基于召回结果和query结合起来构建的prompt效果
result = qa_chain({"query":question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

/tmp/ipykernel_5009/752743503.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query":question_1})


大模型+知识库后回答 question_1 的结果：
南瓜书是一本机器学习领域的书籍，致力于解析西瓜书中比较难理解的公式，以及补充部分公式的推导细节。谢谢你的提问！


In [22]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
王阳明是明代思想家、教育家，提出“心即理”、“知行合一”等哲学观念，被认为是中国哲学史上的重要人物。谢谢你的提问！


In [23]:
# 大模型自己回答的效果
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

# 基于大模型的问答
llm.predict(prompt_template)

/tmp/ipykernel_5009/1089164201.py:6: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict(prompt_template)


'南瓜书是由德国心理学家佛洛伊德（Sigmund Freud）所著的著作，讲述了一个南瓜成为了心理学家。这本书以幽默的方式探讨了心理学的概念和理论，引发了人们对心理学的思考，并成为心理学教育领域中的经典之作。'

In [24]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

# 基于大模型的问答
llm.predict(prompt_template)

'王阳明（1472年 - 1529年），明代著名学者、政治家、军事家。他为心学创立者，提出“知行合一”、“格物致知”等重要观念，对后世影响深远。在政治上主张“致良知”、“治民以仁”，强调内在修养和道德修养的重要性。'

In [25]:
# 添加历史对话的记忆功能
# 记忆(Memory)

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/tmp/ipykernel_5009/592791389.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [31]:
# 对话检索链(ConversationalRetrievvalChain)

from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question":question})
print(result['answer'])

抱歉，我无法回答这个问题。因为提供的文本片段未提供关于具体课程内容和教授这方面知识的信息。您可能需要查阅更多相关内容或直接询问相关领域的专家以获取答案。


In [32]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])


很抱歉，我不知道。
